In [ ]:
import pygame
import random as ran
import os
import time

class block(pygame.sprite.Sprite):
    def __init__(self, x, y, r, g, b, w, h, st, mu, diss, poi):
        super().__init__()
        self.width = w
        self.height = h
        self.image = pygame.Surface([w, h])
        self.color = (r, g, b)
        self.image.fill(self.color)
        self.rect = self.image.get_rect()
        self.rect.x = x
        self.rect.y = y
        self.smallTime = st
        self.music = mu
        self.dissappearTime = diss
        self.pointTime = poi
        
    def flash(self):
        self.color = (ran.randint(0, 255), ran.randint(0, 255), ran.randint(0, 255))
        self.image.fill(self.color)
        
    def expand(self, playMusic):
        self.dissappearTime = 20
        if self.width < 300:
            self.width += 2
            self.height += 2
            self.image = pygame.transform.scale(self.image, (self.width, self.height))
            self.rect = self.image.get_rect(center = (self.rect.x + self.width / 2, self.rect.y + self.height / 2))
        self.smallTime = ran.randint(25, 50)
        if self.pointTime < 60:
            self.pointTime += 2
        if not self.music == 1:
            playMusic.play()
            self.music = 1
    
    def small(self, playMusic):
        self.smallTime = 0
        if self.width > 10:
            self.width -= 2
            self.height -= 2
            self.image = pygame.transform.scale(self.image, (self.width, self.height))
            self.rect = self.image.get_rect(center = (self.rect.x + self.width / 2, self.rect.y + self.height / 2))
        if not self.music == 2:
            playMusic.play()
            self.music = 2

class player(pygame.sprite.Sprite):
    def __init__(self, x, y, r, g, b, w, h):
        super().__init__()
        self.image = pygame.Surface([w, h])
        self.color = (r, g, b)
        self.image.fill(self.color)
        self.rect = self.image.get_rect()
        
    def flash(self):
        self.color = (ran.randint(0, 255), ran.randint(0, 255), ran.randint(0, 255))
        self.image.fill(self.color)
        
def makeThings(count):
    for i in range(count):
        newBlock = block(ran.randint(0, w - 40), ran.randint(0, h - 40), ran.randint(0, 255), ran.randint(0, 255), ran.randint(0, 255), 40, 40, ran.randint(25, 50), 0, 20, 0)
        allSprites.add(newBlock)
        blocks.add(newBlock)
        
def drawbg(x, y, a, b, cx, cy):
    pygame.draw.rect(screen, (ran.randint(0,255), ran.randint(0,255), ran.randint(0,255)), [x, y, a, b], 0)
    if a < 40:
        return
    drawbg(x + 0.13*(cx - x), y + 0.13*(cy - y), (x + a - cx) * 0.87 + cx - x - 0.13*(cx - x), (y + b - cy) * 0.87 + cy - y - 0.13*(cy - y), cx, cy)
    
def print_text(screen, font, x, y, text):
    imgText = font.render(text, True, (ran.randint(0,255), ran.randint(0,255), ran.randint(0,255)), white)
    screen.blit(imgText, (x, y))
        
w = 1080
h = 640
squareW = 40
squareL = 40
score = 0
allSprites = pygame.sprite.Group()
blocks = pygame.sprite.Group()

white = (255, 255, 255)
pygame.init()
size = (w, h)
makeThings(10)
fontSmall = pygame.font.SysFont('SimHei', 40)
fontBig = pygame.font.SysFont('SimHei', 120)
screen = pygame.display.set_mode(size)
clock = pygame.time.Clock()
pygame.display.set_caption("boring game")
riseSound = pygame.mixer.Sound("rise.ogg")
fallSound = pygame.mixer.Sound("fall.ogg")
player = player(0, 0, 0, 0, 0, 40, 40)
allSprites.add(player)
startTime = pygame.time.get_ticks()
run = True
gameOver = False


while run:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            run = False
    
    if not gameOver:
        cat = pygame.mouse.get_pos()
        player.rect.x = cat[0]
        player.rect.y = cat[1]
        player.flash()
        
    player.flash()
    drawbg(0, 0, w, h, cat[0] + 20, cat[1] + 20)
    allSprites.draw(screen)
    
    collide = False
    small = False
    
    if len(blocks) == 0:
        print_text(screen, fontBig, 300, 100, "GAME OVER")
        if not gameOver:
            sec = round((pygame.time.get_ticks() - startTime) / 1000, 1)
        gameOver = True
    
    collideBlocks = pygame.sprite.spritecollide(player, blocks, False)
    
    for i in blocks:
        i.smallTime -= 1
        if i.pointTime > 0:
            i.pointTime -= 1
        if i in collideBlocks:
            i.flash()
            collide = True
            score += round(6 - i.pointTime / 10, 1)
            i.expand(riseSound)
        elif i.smallTime < 0:
            i.small(fallSound)
            if i.width > 13:
                i.flash()
                i.dissappearTime = 20
                small = True
            else:
                i.dissappearTime -= 1
        else:
            i.music = 0
        if i.width < 40:
            score -= (40 - i.width) / max((20 / sec), 0.8)
        if i.width > 200:
            score -= (i.width - 200) / max((20 / sec), 0.8)
        if i.dissappearTime < 0:
            allSprites.remove(i)
            blocks.remove(i)
            
    
    if not collide:
        riseSound.stop()
    if not small:
        fallSound.stop()
    
    if not gameOver:
        sec = round((pygame.time.get_ticks() - startTime) / 1000, 1)
        score += len(blocks) / 2
    if score < 0:
        score = 0
    print_text(screen, fontSmall, 0, 0, str(sec) + " s")
    print_text(screen, fontSmall, 0, 50, "Score: " + str(round(score, 1)))
    
    clock.tick(20)
    pygame.display.flip()

    
pygame.display.quit()
pygame.quit()
os._exit(0)
    

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
